<a href="https://colab.research.google.com/github/kamalahmadov474/GPT-2-LLM-For-Industrial-Automation-Systems/blob/main/LLM_Fine_Tuned_Chatbot(Filling_Tank).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Step 1: Install necessary libraries

In [ ]:
pip install transformers datesets accelerate

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement datesets (from versions: none)
ERROR: No matching distribution found for datesets

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


STEP 2: Prepare the dataset for Fine-Tuning

In [ ]:
from datasets import Dataset
data = {
    "instruction": [
        "Start filling the tank for 20 seconds",
        "Discharge the tank for 15 seconds",
        "Is the tank currently filling?"
    ],
    "output": [
        "write_coil(0, True)",                         # Fill
        "write_coil(1, True)",                         # Discharge
        "read_discrete_input(1)"                     # Filling (Input 1)
    ]
}
dataset = Dataset.from_dict(data)
# Normalize function
def normalize(text):
    return re.sub(r"\s+", "", text.lower())

c:\Users\TUF\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


STEP 3: LLM Fine-Tune the Model

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer

model_name = "gpt2"  # Use a small model for quick testing
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Set pad_token to eos_token
tokenizer.pad_token = tokenizer.eos_token

def preprocess(example):
    # Encode with padding and truncation
    inputs = tokenizer(
        f"User: {example['instruction']}\nBot: {example['output']}",
        truncation=True,
        padding="max_length",  # Add padding
        max_length=512  # Set a maximum length
    )

    inputs['labels'] = inputs['input_ids'].copy()
    return inputs

# Apply the preprocessing to the dataset
tokenized_dataset = dataset.map(preprocess, remove_columns=dataset.column_names)

training_args = TrainingArguments(
    output_dir="./modbus-llm",
    per_device_train_batch_size=2,
    num_train_epochs=5,
    logging_steps=10,
    save_steps=100,
    fp16=True,
    overwrite_output_dir=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
)

trainer.train()

Map: 100%|██████████| 3/3 [00:00<00:00, 71.80 examples/s]
C:\Users\TUF\AppData\Local\Temp\ipykernel_20364\1849354702.py:35: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
c:\Users\TUF\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
10,2.241000


TrainOutput(global_step=10, training_loss=2.2410154342651367, metrics={'train_runtime': 44.6814, 'train_samples_per_second': 0.336, 'train_steps_per_second': 0.224, 'total_flos': 3919380480000.0, 'train_loss': 2.2410154342651367, 'epoch': 5.0})

STEP 4: Connect to CODESYS / Factory I/O via Modbus

In [ ]:
import re
from pymodbus.client import ModbusTcpClient

# Attempt to connect to Modbus server
client = ModbusTcpClient('127.0.0.1', port=502)
if not client.connect():
    raise ConnectionError("❌ Failed to connect to Modbus server at 127.0.0.1:502")

print("✅ Modbus connection established.")
import re
# Get user instruction
# Get user input
instruction = input("🧠 Enter your instruction: ").strip()
normalized_instruction = normalize(instruction)

# Find matching instruction
matched_idx = None
for idx, inst in enumerate(dataset["instruction"]):
    if normalize(inst) == normalized_instruction:
        matched_idx = idx
        break

if matched_idx is not None:
    command_str = dataset["output"][matched_idx]
    print(f"🤖 Matched instruction: {dataset['instruction'][matched_idx]}")
    print(f"⚙️ Executing command(s): {command_str}")

    commands = [cmd.strip() for cmd in command_str.split(";")]
    for cmd in commands:
        match = re.match(r"write_coil\((\d+),\s*(True|False)\)", cmd, re.IGNORECASE)
        if match:
            address = int(match.group(1))
            value = match.group(2).lower() == "true"
            result = client.write_coil(address, value)
            print(f"✅ Coil {address} written to {value}: {result}")
        else:
            print(f"ℹ️ Skipped: '{cmd}' is not a write command")
else:
    print("❌ Instruction not recognized.")